# Welcome To The Notebook

### Task 1 - Set up the project environment

In [ ]:
!pip install openai==1.7.2 python-dotenv

Importing modules

In [ ]:
import pandas as pd
import os, time
from openai import OpenAI
from dotenv import load_dotenv
import json
import matplotlib.pyplot as plt

print("Modules are imported.")

Setting up the OpenAI API:

* Prepare a .env file to store the OpenAI API key.
* Uploading the .env file to our colab environment
* Load the API key and setup the API

In [ ]:
load_dotenv('apikey.env.txt')

APIKEY=os.getenv('APIKEY')

Creating OpenAI Client

In [ ]:
client=OpenAI(
    api_key=APIKEY
)
client

### Task 2 - Prepare the training data

Loading the provided `Customer Complaints.csv`



In [ ]:
training_data=pd.read_csv('Customer Complaints.csv')
training_data

**Converting the Complaints records to json**

To be able to use the data for the fine-tuning purpose, we first need to convert each row of the dataframe into the following format:

<pre>
<code>
{
  <span style="color: blue;">"messages"</span>: [
    {
      <span style="color: blue;">"role"</span>: <span style="color: red;">"system"</span>,
      <span style="color: blue;">"content"</span>: "<span style="color: green;">Providing context about the user's prompt.
                  It may include information about the task,
                  instructions, or background details relevant
                  to the conversation.</span>"
    },
    {
      <span style="color: blue;">"role"</span>: <span style="color: red;">"user"</span>,
      <span style="color: blue;">"content"</span>: "<span style="color: green;">the prompt or input provided by the user,
                  which typically initiates the conversation with the assistant.</span>"
    },
    {
      <span style="color: blue;">"role"</span>: <span style="color: red;">"assistant"</span>,
      <span style="color: blue;">"content"</span>: "<span style="color: green;">The desired response or output generated by
                  the assistant in response to the user's prompt.</span>"
    }
  ]
}
</code>
</pre>

Let's get started!

Let's define a method that get's a row of the dataframe and convert it into the json format

In [ ]:
def save_as_json(row):

  system_content = """
      Given a customer complaint text, extract and return the following information in json (dict) format:
      - Topic: The product/department that the customer has a complaint about.
      - Problem: A two or three-word description of what exactly the problem is.
      - Customer_Dissatisfaction_Index: is a number between 0 and 100 showing
             how angry the customer is about the problem.
  """

  formatted_data = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": row.Complaints},
            {"role": "assistant", "content": row.Details}
        ]
      }

  with open("training_data.json", "a") as json_file:
        json.dump(formatted_data, json_file)
        json_file.write("\n")

Now let's use of this method to generate the `training_data.json`

In [ ]:
for index, row in training_data.iterrows():
  save_as_json(row)

### Task 3 - Fine-tune GPT 3.5 based on our training data

Let's import the json file we prepared as our training data

In [ ]:
data_file=client.files.create(
    file=open('training_data.json','rb'),
    purpose='fine-tune',

)
data_file

Create the Fine Tuning Job

In [ ]:
fine_tuning_job=client.fine_tuning_jobs.create(
    training_file=data_file.id,
    model='gpt-3.5-turbo',
    hyperparameters={
        "n_epochs":'auto'

    }
)
fine_tuning_job

Let's retrieve the state of the fine-tune

In [ ]:
while True:
  time.sleep(2)
  retrieved_jobs=client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
  status=retrieved_jobs.status
  print(status)

  if status=="succeeded":
    print("The job is done!")
    break


### Task 4 - Evaluate model

Let's retrieve the event messages to check out the learning process of our fine-tuning job.

In [ ]:
events=list(client.fine_tuning_jobs.list_events(fine_tuning_job_id=retrieved_jobs.id,limit=100).data)
for e in events:
  print(e.message)



Let's extract the training loss in each learning step

In [ ]:
steps=[]
train_loss=[]

for e in events:
  if(e.data):
    steps.append(e.data['step'])
    train_loss.append(e.data['train_loss'])

print(steps)
print(train_loss)


Use a line chart to visualize the train_loss in each step

In [ ]:
plt.plot(steps,train_loss,maker='o',linestyle='-')

### Task 5 - Deploy our model

Let's take a look at `retrieved_jobs` again

In [ ]:
myLLM=retrieved_jobs.fine_tune_model
print(myLLM)

Defining a method to extract information from a given user complaint using a specific LLM and return the results.

In [ ]:
def extract_details(user_complaint, model_name):
    """
    This function extracts information from a given user complaint using a specific LLM (Large Language Model).

    Parameters:
    user_complaint (str): The text of the user's complaint.
    model_name (str): The name of the specific LLM model to use for extraction.
    """

    system_content = """
        Given a customer complaint text, extract and return the following information in JSON (dict) format:
        - Topic
        - Problem
        - Customer_Dissatisfaction_Index
    """

    # Generate a response using the specified model and the user's complaint
    response = client.chat.completions.create(
        model = model_name,
        messages=[
            {"role": "system", "content": system_content},  # System content explaining the expected output
            {"role": "user", "content": user_complaint}  # User's complaint passed as content
        ]
    )

    # Return the content of the generated response
    return response.choices[0].message.content


Let's use our fine-tuned model to extract the details for the following user complaint:

*TV channels keep disappearing from my subscription! What's going on? Extremely annoyed with this service!*

In [ ]:
complaint="TV channels keep disappearing from my subscription! What's going on? Extremely annoyed with this service!"
extract_details(complaint,myLLM)

Let's test our `GPT-4` model with the same user complaint

In [ ]:
extract_details(complaint,'gpt-4')

Let's try for the following complaint:

*Line is down! It is really annoying!*

In [ ]:
complaint="Line is down! It is really annoying!"
extract_details(complaint,myLLM)

Now let's compare the results from GPT-4

In [ ]:
extract_details(complaint,'gpt-4')

We can see that our model, which is trained on our dataset, provides better answers compared to GPT-4. Our model is fine-tuned based on our data and is familiar with the different edge cases and the context of our dataset.